# Simulation and Noise Models

In [16]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:
import sys
sys.path.insert(0, '../')

We can set up a noise model, following the [Qiskit textbook](https://qiskit.org/textbook/ch-quantum-hardware/error-correction-repetition-code.html#Correcting-errors-in-qubits). We define our noise model to have equal chances of X and Z:

In [18]:
from qiskit.providers.aer.noise import NoiseModel
from qiskit.providers.aer.noise.errors import pauli_error, depolarizing_error

def get_noise(p_err):

    error_gate1 = pauli_error([('X', p_err/2), ('Z', p_err/2), ('I', 1 - p_err)])
    error_gate2 = error_gate1.tensor(error_gate1)

    noise_model = NoiseModel()
    noise_model.add_all_qubit_quantum_error(error_gate1, "measure") # measurement error is applied to measurements
    noise_model.add_all_qubit_quantum_error(error_gate1, ["u1", "u2", "u3"]) # single qubit gate error is applied to x gates
    noise_model.add_all_qubit_quantum_error(error_gate2, ["cx"]) # two qubit gate error is applied to cx gates
        
    return noise_model

In [19]:
from qiskit import execute, Aer
from circuits import SurfaceCode
from fitters import GraphDecoder

We set up a test harness to run through some examples:

In [9]:
def full_loop(code, decoder1, decoder2, noise_model=None):
    # noise_model = get_noise(0.01)
    counts = execute(code.circuit['0'], Aer.get_backend('qasm_simulator'), shots=1).result().get_counts()
    X_errors, Z_errors = code.extract_nodes(code.process_results(counts))

    print("Raw readout:", counts)
    print("Parsed X:", X_errors)
    print("Parsed Z:", Z_errors)

    # X_errors = [node for node in X_errors if node[0] != -2]
    # Z_errors = [node for node in Z_errors if node[0] != -2]

    for decoder in (decoder1, decoder2):
        if X_errors:
            error_graph, _ = decoder.make_error_graph(X_errors, "X")
            matching_graph = decoder.matching_graph(error_graph,'X')
            matches = decoder.matching(matching_graph,'X')
            _, paths = decoder2.make_error_graph(X_errors, "X")
            flips_x = decoder.calculate_qubit_flips(matches, paths,'X')
        else:
            flips_x = {}

        if Z_errors:
            error_graph, _ = decoder.make_error_graph(Z_errors, "Z")
            matching_graph = decoder.matching_graph(error_graph,'Z')
            matches = decoder.matching(matching_graph,'Z')
            _, paths = decoder2.make_error_graph(X_errors, "Z")
            flips_z = decoder.calculate_qubit_flips(matches, paths,'Z')
        else:
            flips_z = {}

        print("----------------")
        print("Decoder results" + (" (sim):" if decoder == decoder1 else ":"))
        print(decoder.net_qubit_flips(flips_x, flips_z))

In [6]:
code = SurfaceCode(3, 1)
decoder1 = GraphDecoder(3, 1, simulation=True)
decoder2 = GraphDecoder(3, 1)

We can compare `decoder1`, the syndrome graph from simulation, with `decoder2`, the analytic graph:

In [10]:
for i in range(10):
    print("====Run", i, "====")
    full_loop(code, decoder1, decoder2)
    print("=============", "\n")

====Run 0 ====
Raw readout: {'110110110 00000100': 1}
Parsed X: [(0, 1.5, 0.5)]
Parsed Z: []
----------------
Decoder results (sim):
Physical Qubit: (1.0, 0.0)
Error: X at time: 0
{(1.0, 0.0): array([[0, 1],
       [1, 0]])}
----------------
Decoder results:
Physical Qubit: (1.0, 0.0)
Error: X at time: 0
{(1.0, 0.0): array([[0, 1],
       [1, 0]])}

====Run 1 ====
Raw readout: {'110110101 10000100': 1}
Parsed X: [(0, -0.5, 0.5), (0, 1.5, 0.5)]
Parsed Z: []
----------------
Decoder results (sim):
Physical Qubit: (0.0, 1.0)
Error: X at time: 0
Physical Qubit: (1.0, 1.0)
Error: X at time: 0
{(0.0, 1.0): array([[0, 1],
       [1, 0]]), (1.0, 1.0): array([[0, 1],
       [1, 0]])}
----------------
Decoder results:
Physical Qubit: (0.0, 1.0)
Error: X at time: 0
Physical Qubit: (1.0, 1.0)
Error: X at time: 0
{(0.0, 1.0): array([[0, 1],
       [1, 0]]), (1.0, 1.0): array([[0, 1],
       [1, 0]])}

====Run 2 ====
Raw readout: {'101101101 00000100': 1}
Parsed X: [(0, 1.5, 0.5)]
Parsed Z: []
-----

And now run with some noise:

In [ ]:
for i in range(10):
    print("====Run", i, "====")
    full_loop(code, decoder1, decoder2, noise_model=get_noise(0.5))
    print("=============", "\n")